# Task-1:
Choose any one deep learning algorithm of your choice and try to
implement it from scratch.
Important points:
<ul>
● You can use modules like JAX, Numpy, etc. for your
implementations.<br>
● You should be able to explain the mathematical concept
behind your implementation.<br>
 </ul>
Judging Criteria:<br>
● Structure of your code.<br>
● Math and code implementation.<br>
<br>
Take the given Implementation as a reference: Link <br>

Note: For your reference, the link attached herewith shows implementations
of a few ML algorithms. However, the task is to implement DL algorithms
from scratch.<br>

A few of the DL algorithms you can implement but are not limited
to are Multi-layer perceptron, Convolutional Neural Nets,
Recurrent Neural Nets, etc.